# core

> This is where users start to interact with the underlying LLM and agents.

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
# A chatbot that allows users to interact with an LLM
import gradio as gr

def taskRouter(message, history):
    if message.endswith("?"):
        return "Good question!"
    elif "Hello" in message:
        return "Hello! How can I help?"
    else:
        return "Let me work on that...."

def vote(data: gr.LikeData):
    if data.liked:
        print("You upvoted this response: " + data.value["value"])
    else:
        print("You downvoted this response: " + data.value["value"])

with gr.Blocks() as sciChat:
    chatbot = gr.Chatbot(height=300,placeholder="<strong>Your Personal Science AI Assistant</strong><br>Ask Me Anything")
    chatbot.like(vote, None, None)
    gr.ChatInterface(
        fn=taskRouter,
        chatbot=chatbot,
        textbox=gr.Textbox(placeholder="How can I help?", container=False, scale=7),
        title="SciAI Assistant",
        description="Ask me any question or carry out a task",
        theme="soft",
        examples=["Hello", "What's new last week in neuroscience?", "I need help analyze some data"],
        cache_examples=True
    )
    
sciChat.launch(share=True)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()